In [50]:
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [51]:
def extract_doc(doc):
    f = open(doc,encoding="utf-8")
    data = f.read()
    word_tokens = word_tokenize(data)
    stop_words = list(set(stopwords.words('english')))
    filtered_sentence = []
    stopwords_list = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
        else:
            stopwords_list.append(w)
    return filtered_sentence
def extract_q(q):
    data = q
    word_tokens = word_tokenize(data)
    stop_words = list(set(stopwords.words('english')))
    filtered_sentence = []
    stopwords_list = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
        else:
            stopwords_list.append(w)
    return filtered_sentence

In [52]:
d1 = extract_doc("sixyears.txt")
d2 = extract_doc("traintonowhere.txt")
d3 = extract_doc("whatdreams.txt")
d4 = extract_doc("gettingsaucy.txt") 

In [36]:
#Bag of words
distinct_words = list(set(d1+d2+d3+d4)) 
import pandas as pd
def Bag_of_words(word_list): 
    word_l = []
    for word in distinct_words:
        if (word in word_list) == True:
            word_l.append(word_list.count(word))
        else:
            word_l.append(0)
    return word_l
df = pd.DataFrame()
df["words"] = distinct_words
df["d1"] = Bag_of_words(d1)
df["d2"] = Bag_of_words(d2)
df["d3"] = Bag_of_words(d3)
df["d4"] = Bag_of_words(d4)

print(df)

         words  d1  d2  d3  d4
0       parsed   0   1   0   0
1      setting   0   0   1   0
2        happy   0   0   1   0
3        steak   0   0   0   2
4         used   0   2   0   0
..         ...  ..  ..  ..  ..
895     enjoys   0   0   1   0
896  grappling   1   0   0   0
897    lasting   0   1   0   0
898    growing   0   1   0   1
899  10-minute   0   0   0   1

[900 rows x 5 columns]


In [37]:
#TF
#We use the log function
# tf = 1 + log(1+log(nij))
import math
def TF(word_list):
    word_l = []
    for word in distinct_words: 
        if (word in word_list) == True:
            word_l.append(1+math.log(1+math.log(word_list.count(word))))
        else:
            word_l.append(0)
    return word_l
tf = pd.DataFrame()
tf["words"] = distinct_words
tf["d1"] = TF(d1)
tf["d2"] = TF(d2)
tf["d3"] = TF(d3)
tf["d4"] = TF(d4)

print(tf)

         words   d1        d2   d3        d4
0       parsed  0.0  1.000000  0.0  0.000000
1      setting  0.0  0.000000  1.0  0.000000
2        happy  0.0  0.000000  1.0  0.000000
3        steak  0.0  0.000000  0.0  1.526589
4         used  0.0  1.526589  0.0  0.000000
..         ...  ...       ...  ...       ...
895     enjoys  0.0  0.000000  1.0  0.000000
896  grappling  1.0  0.000000  0.0  0.000000
897    lasting  0.0  1.000000  0.0  0.000000
898    growing  0.0  1.000000  0.0  1.000000
899  10-minute  0.0  0.000000  0.0  1.000000

[900 rows x 5 columns]


In [43]:
#IDF
idf = pd.DataFrame()
def IDF():
    l = []
    D = [d1,d2,d3,d4]
    for i in distinct_words:
        d = 0
        for j in D:
            if i in j:
                d+=1
        l.append(math.log(1+4/d))
    return l
 
idf["word"] = distinct_words
idf["relevance"] = IDF()

print(idf) 

          word  relevance
0       parsed   1.609438
1      setting   1.609438
2        happy   1.609438
3        steak   1.609438
4         used   1.609438
..         ...        ...
895     enjoys   1.609438
896  grappling   1.609438
897    lasting   1.609438
898    growing   1.098612
899  10-minute   1.609438

[900 rows x 2 columns]


In [44]:
#TF_IDF
import numpy as np
tfidf = pd.DataFrame()
tfidf["words"] = distinct_words
n1 = np.array(tf.d1)
n2 = np.array(tf.d2)
n3 = np.array(tf.d3)
n4 = np.array(tf.d4)

x = np.array(idf.relevance)
tfidf["d1"] = n1*x
tfidf["d2"] = n2*x
tfidf["d3"] = n3*x
tfidf["d4"] = n4*x

print(tfidf)

         words        d1        d2        d3        d4
0       parsed  0.000000  1.609438  0.000000  0.000000
1      setting  0.000000  0.000000  1.609438  0.000000
2        happy  0.000000  0.000000  1.609438  0.000000
3        steak  0.000000  0.000000  0.000000  2.456950
4         used  0.000000  2.456950  0.000000  0.000000
..         ...       ...       ...       ...       ...
895     enjoys  0.000000  0.000000  1.609438  0.000000
896  grappling  1.609438  0.000000  0.000000  0.000000
897    lasting  0.000000  1.609438  0.000000  0.000000
898    growing  0.000000  1.098612  0.000000  1.098612
899  10-minute  0.000000  0.000000  0.000000  1.609438

[900 rows x 5 columns]


In [45]:
#normalized TDIDF for Query and document
tfidf.d1/=len(distinct_words)
tfidf.d2/=len(distinct_words)
tfidf.d3/=len(distinct_words)
tfidf.d4/=len(distinct_words)
print(tfidf) 

         words        d1        d2        d3        d4
0       parsed  0.000000  0.001788  0.000000  0.000000
1      setting  0.000000  0.000000  0.001788  0.000000
2        happy  0.000000  0.000000  0.001788  0.000000
3        steak  0.000000  0.000000  0.000000  0.002730
4         used  0.000000  0.002730  0.000000  0.000000
..         ...       ...       ...       ...       ...
895     enjoys  0.000000  0.000000  0.001788  0.000000
896  grappling  0.001788  0.000000  0.000000  0.000000
897    lasting  0.000000  0.001788  0.000000  0.000000
898    growing  0.000000  0.001221  0.000000  0.001221
899  10-minute  0.000000  0.000000  0.000000  0.001788

[900 rows x 5 columns]


In [53]:
def cos_sim(d1, d2):
  l1 = list(d1)
  l2 = list(d2)
  c=0
  for i in range(len(tfidf.words)): 
          c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  return cosine

In [54]:
#finding cosine similarity
print("Cosine Similarity between doc 1 and doc 2 is ", cos_sim(tf.d1, tf.d2))
print("Cosine Similarity between doc 1 and doc 3 is ", cos_sim(tf.d1, tf.d3))
print("Cosine Similarity between doc 1 and doc 4 is ", cos_sim(tf.d1, tf.d4))
print("Cosine Similarity between doc 2 and doc 3 is ", cos_sim(tf.d2, tf.d3))
print("Cosine Similarity between doc 2 and doc 4 is ", cos_sim(tf.d2, tf.d4))
print("Cosine Similarity between doc 3 and doc 4 is ", cos_sim(tf.d3, tf.d4))

Cosine Similarity between doc 1 and doc 2 is  0.2279894819604541
Cosine Similarity between doc 1 and doc 3 is  0.23593072757345418
Cosine Similarity between doc 1 and doc 4 is  0.22444947517665506
Cosine Similarity between doc 2 and doc 3 is  0.21741686092162324
Cosine Similarity between doc 2 and doc 4 is  0.23624882983298426
Cosine Similarity between doc 3 and doc 4 is  0.21381306271857836
